In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch, torchvision 
from PIL import Image

In [3]:
df_infos2 = pd.read_csv('../saved/infos2.csv')

In [ ]:
def import_chn_imgs(chn):
    st = time.time()
    data_imgs = []
    
    #This line is the strange hack
    print(' ', end='', flush=True)
    
    for i in tqdm(range(len(df_infos2)), desc=str(chn), mininterval=1): #   
        with Image.open('../saved/channel_imgs/{}/'.format(chn)+'/'+str(i)+'.png') as img:
            data_imgs.append(img.copy())
    return data_imgs

import time
st = time.time()

chn_imgs = [import_chn_imgs(chn) for chn in range(12)]

print("Time passed:", int(time.time()-st), "min")

In [ ]:
plt.imshow(chn_imgs[1][0])

In [ ]:
from global_vars import labels

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
import os
class MyImageMultichannelDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, infos, channel_imgs):
        """
                
        channel_imgs = chn -> array of PImage
        """
        self.infos = infos
        self.channel_imgs = channel_imgs
        self.transform =  transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                         ])

    def __len__(self):
        return len(self.infos)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        images = [self.channel_imgs[i][idx] for i in range(12)]
        info_labels = self.infos.iloc[idx][labels]
        sample =([self.transform(image) for image in images], torch.Tensor(info_labels.astype(int)))

        return sample
    
image_datasets = MyImageMultichannelDataset(df_infos2, chn_imgs)

In [ ]:
len(chn_imgs[1])

In [ ]:
imgs0, label0 = image_datasets[0]

In [ ]:
# dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=4,
#                                              shuffle=True, num_workers=4)
# dataset_sizes = len(image_datasets)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
from torch import nn
from torchvision import models
import torch.nn.functional as F
import torch.optim as optim

def load_cwt_nn_model(device, model_saved_path='../saved/modelCWTFullBalanced/CWTNetFull0_model.dict', 
                      freeze=True):
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 9)
    model.load_state_dict(torch.load(model_saved_path, map_location=device))
    
    if freeze:
        for param in model.parameters():
            param.requires_grad = False
    # change the last output to 9 classes
    model.fc = nn.Linear(num_ftrs, 1000)
    
    # load saved model
    
    model.eval()
    model.to(device)
    return model

class MultiCWTNet(nn.Module):
    def __init__(self, device, verbose=False):
        super(MultiCWTNet, self).__init__()
        
        self.chn_resnets = [load_cwt_nn_model(device) for i in range(12)]
        num_ftrs = self.chn_resnets[0].fc.out_features
        self.fc1 = nn.Linear(num_ftrs*12, 9)
        self.fc2 = nn.Linear(9, 9)
        self.verbose = verbose
        
    def forward(self, xs):
        x = [self.chn_resnets[i](xs[i]) for i in range(12)]
        
        x = torch.cat(x, 1)
        if self.verbose:
            print('0: ', x.shape, x.device)
            
        x = self.fc1(x)
        if self.verbose:
            print('2: ', x.shape)
            
        x = F.relu(   x )     
        if self.verbose:
            print('3: ', x.shape)
            
        x = self.fc2(x)
        
        if self.verbose:
            print('4: ', x.shape)
        return x

# model = MultiCWTNet(device, verbose=True)
# model.to(device)
# for X,y in dataloaders:
#     xs = [x.to(device) for x in X]
#     model.forward(xs)
#     break

In [ ]:

# def make_weights_for_balanced_classes(ys, weight_per_class):                                                                                          
#     weight = [0] * len(ys)                                              
#     for idx, y in enumerate(ys):    
#         for i, val in enumerate(y):
#             weight[idx] += weight_per_class[i]                           
#     return weight  


In [ ]:
from torchvision import datasets, models, transforms
from myeval import binary_acc
import torch.optim as optim
from torch.optim import lr_scheduler
from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import GroupKFold
import time

st = time.time()
patience = 20
kf = GroupKFold(10)
batch_size=20

saved_dir = '../saved/modelMultiCWTFull'
y = df_infos2[labels].astype(int)

for i, (train_idx, test_idx) in enumerate(kf.split(df_infos2, y, df_infos2['ptID'])):
    
    
#     df_y_train = df_infos2.iloc[train_idx][labels].to_numpy().astype(int)
#     weights = make_weights_for_balanced_classes(df_y_train, 1.0/np.sum(df_y_train,axis=0))

#     per_weights = torch.Tensor(weights).to(device)
    #sampler = torch.utils.data.sampler.WeightedRandomSampler(per_weights, len(train_idx))

    trainDataset = torch.utils.data.Subset(image_datasets, train_idx)
    testDataset = torch.utils.data.Subset(image_datasets, test_idx)
    
    trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle = True, pin_memory=True)#sampler = sampler)
    testLoader = torch.utils.data.DataLoader(testDataset, batch_size = batch_size, shuffle = False, pin_memory=True)

    model = MultiCWTNet(device, verbose=False)
    model.to(device)
    
    
    # load saved model
#     model_saved_path = saved_dir + 'CWTNetFull0_model.dict'
#     model.load_state_dict(torch.load(model_saved_path))
    

    
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) #
    # Decay LR by a factor of 0.1 every 100 epochs
    scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    class_weights = np.ones(9) * 2
    class_weights = torch.Tensor(class_weights).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)

    losses_train = []
    losses_test = []

    avg_losses_train = []
    avg_losses_test = []


    early_stopping = EarlyStopping(patience, verbose=False, 
                                  saved_dir=saved_dir, 
                                   save_name='MutliCWTNetFull'+str(i))
    epoch = 0
    auroc = 0
    auprc = 0
    accuracy = 0
    fmeasure = 0
    fbeta = 0
    gbeta = 0
    for epoch in range(1000):
        
        model.train()
        output_trains = []
        y_trains = []
        for X_train, y_train in tqdm(trainLoader):
            y_train = y_train.to(device)
            X_train_chns = [x.to(device) for x in X_train]
            optimizer.zero_grad()
            output_train = model(X_train_chns)
            
            loss_train = criterion(output_train, y_train)
            losses_train.append(loss_train.item())
            loss_train.backward()
            optimizer.step()
            
            output_trains.append(output_train.cpu())
            y_trains.append(y_train.cpu())

        scheduler.step()
        
        avg_loss_train = np.average(losses_train)
        avg_losses_train.append(avg_loss_train)
        
        output_tests = []
        y_tests = []
        with torch.no_grad():
            model.eval()
            
            for X_test, y_test in testLoader:  
                y_test = y_test.to(device)
                X_test_chns = [x.to(device) for x in X_test]
                output_test = model(X_test_chns)
                
                loss_test = criterion(output_test, y_test)
                losses_test.append(loss_test.item())
                
                output_tests.append(output_test.cpu())
                y_tests.append(y_test.cpu())
                
            avg_loss_test = np.average(losses_test)
            avg_losses_test.append(avg_loss_test)

       
        
        output_trains = torch.cat(output_trains, axis=0)
        y_trains = torch.cat(y_trains, axis=0)
        acc, fmeasure, fbeta, gbeta, auroc, auprc = binary_acc(output_trains, y_trains)

        output_tests = torch.cat(output_tests, axis=0)
        y_tests = torch.cat(y_tests, axis=0)
        acc2, fmeasure2, fbeta2, gbeta2, auroc2, auprc2 = binary_acc(output_tests, y_tests)

        print('S{}/{} {:.2f} min|\nTrain Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, auroc: {:.3f}, auprc: {:.3f} |\nValid Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, auroc: {:.3f}, auprc: {:.3f}\n '.format(
            i, epoch, (time.time()-st)/60,
            avg_loss_train, acc, fmeasure, fbeta, gbeta, auroc, auprc, 
            avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, auroc2, auprc2))
#         print('S{}/{} {:.2f} min|\nTrain Loss: {:.6f} |Valid Loss: {:.6f} '.format(
#              i, epoch, (time.time()-st)/60, avg_loss_train, avg_loss_test))

        early_stopping(avg_loss_test, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break
            
    output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc2,auprc2,acc2,fmeasure2,fbeta2,gbeta2)
    print(output_string)     
    with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
        f.write(output_string)

    avg_losses_train = np.array(avg_losses_train)
    avg_losses_test = np.array(avg_losses_test)
    
    np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
    np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)
    